In [1]:
#install  google-api-python-client

In [1]:
import store_to_vdb

# from apiclient.discovery import build
# from googleapiclient.discovery import build

In [2]:
captions = store_to_vdb.playlist_to_captions("PLUl4u3cNGP619EG1wp0kT-7rDE_Az5TNd")
len(captions)

15

In [3]:
captions = store_to_vdb.captions_processing(captions)
print(captions[0]['video_captions'][0]['text'])
for i in range(len(captions)):
    print(len(captions[i]['video_captions']))

The following content is provided under a Creative Commons license.
358
430
760
447
456
479
426
448
608
713
762
495
421
524
398


In [4]:
# import time
# import cohere


# def captions_to_embeddings(videos_captions):
#     api_key = 'SrLBFEol9tRK7n7LY9FRRcHuN9QT5MLIzLmBSziT'
#     # Create and retrieve a Cohere API key from os.cohere.ai
#     co = cohere.Client(api_key)
#     all_embeds = []
#     no_texts = 0
#     for captions in videos_captions :
#         embeds = []
#         texts = []
#         for caption in captions['video_captions'] :
#             dic = {}
#             dic['start'] = caption['start']
#             dic['video_id'] = captions['video_id']
#             texts.append(caption['text'])
#             dic['embedding'] = []  
#             embeds.append(dic)
#         if(len(texts) > 1600):
#             #call on video captions embedding
# #             video_embeddings = 
#             pass
#         else:
#             if((no_texts+len(texts)) > 1600 ):
#                 no_texts = 0 
#                 print("sleep")
#                 time.sleep(60)
#             no_texts += len(texts)
#             video_embeddings = co.embed(texts,model="small").embeddings
#             print(len(texts))
        
#         for i in range(len(embeds)):
#             embeds[i]['embedding'] =  video_embeddings[i]
        
#         all_embeds.append(embeds)
          
#     return all_embeds


In [5]:
emds = store_to_vdb.captions_to_embeddings(captions)
len(emds)

15

In [12]:
import numpy as np
import pinecone
# import cohere
# co = cohere.Client("SrLBFEol9tRK7n7LY9FRRcHuN9QT5MLIzLmBSziT")

# query = "hello world"

# index = pinecone.Index("first-index")
# # create the query embedding
# xq = co.embed(
#     texts=[query],
#     model='small',
#     truncate='LEFT'
# ).embeddings[0]

# example = [[ {"start":6 , "video_id":"C1lhuz6pZC0" , "embedding":xq,"text": query } ]]




def Piencone_indexing(collection):    #    -> [[{}]]  -> [[ {start:--- , video_id:--- , embedding:--- }   ,]]
    
    PIENCONE_API_KEY = "3d2006de-95b3-4e7d-9ec1-54133c34001e"
    
    pinecone.init(PIENCONE_API_KEY, environment='us-west1-gcp')
    index_name = 'first-index'
    print('============1===============')
    
    shape = np.array(collection[0][0]["embedding"]).shape       # (1024,)

    # if the index does not exist, we create it
    if index_name not in pinecone.list_indexes():
        pinecone.create_index(
            index_name,
            dimension=shape[0],
            metric='dotproduct'
        )
    # connect to index
    index = pinecone.Index(index_name)
    print('============2===============')
    
    last_id = 0 
    for x in range(len(collection)):
        list_dic_data = collection[x]
        print(len(collection[x]))
        num_embeds = len(list_dic_data)
        print(shape)

        batch_size = 128

        ids = [str(i) for i in range( last_id , last_id+ num_embeds + 1)]
        last_id = last_id + num_embeds + 1
        # create list of metadata dictionaries
        embeds = []
        meta_data = []
        
        for dic_data in list_dic_data:
            
            meta = {}
            for key in dic_data.keys():
                if key == 'embedding':
                    embeds.append(dic_data[key])
                else:
                    meta[key] = dic_data[key]
            meta_data.append(meta)

        print(meta_data[0]["video_id"] , " ",len(meta_data))


        # create list of (id, vector, metadata) tuples to be upserted
        to_upsert = list(zip(ids, embeds, meta_data))
#         to_upsert = list(zip(ids, embeds))
        for i in range(0, shape[0]+1, batch_size):
            i_end = min(i+batch_size, shape[0])
            a = to_upsert[i:i_end]
            if(len(a) == 0):
                print("empty")
                break

            index.upsert(vectors=a)

        # let's view the index statistics
        print(index.describe_index_stats())

    return None

In [13]:
Piencone_indexing(emds)

============1===============
============2===============
358
(1024,)
C1lhuz6pZC0   358
empty
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 358}},
 'total_vector_count': 358}
430
(1024,)
uK5yvoXnkSk   430
empty
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 788}},
 'total_vector_count': 788}
760
(1024,)
V_TulH374hw   760
empty
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1548}},
 'total_vector_count': 1548}
447
(1024,)
-1BnXEwHUok   447
empty
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1995}},
 'total_vector_count': 1995}
456
(1024,)
6wUD_gp5WeE   456
empty
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2451}},
 'total_vector_count': 2451}
479
(1024,)
OgO1gpXSUzU   479
empty
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2930}},
 'total_vector_count': 2930}
426
(1024,)
rUxP7TM8-w

In [20]:
import cohere
co = cohere.Client("SrLBFEol9tRK7n7LY9FRRcHuN9QT5MLIzLmBSziT")

query = "hello world"

index = pinecone.Index("first-index")
# create the query embedding
xq = co.embed(
    texts=[query],
    model='small',
    truncate='LEFT'
).embeddings

print(np.array(xq).shape)

# query, returning the top 5 most similar results
res = index.query(xq, top_k=5, include_metadata=True)

(1, 1024)


In [21]:
res

{'matches': [{'id': '0',
              'metadata': {'start': 6.0,
                           'text': 'hello world',
                           'video_id': 'C1lhuz6pZC0'},
              'score': 1905.05444,
              'sparseValues': {},
              'values': []},
             {'id': '460',
              'metadata': {'start': 2144.35, 'video_id': 'vIFKGFl1Cn8'},
              'score': 921.666809,
              'sparseValues': {},
              'values': []},
             {'id': '324',
              'metadata': {'start': 1621.628, 'video_id': 'vIFKGFl1Cn8'},
              'score': 910.555786,
              'sparseValues': {},
              'values': []},
             {'id': '668',
              'metadata': {'start': 2628.49, 'video_id': 'V_TulH374hw'},
              'score': 900.626099,
              'sparseValues': {},
              'values': []},
             {'id': '248',
              'metadata': {'start': 1277.2, 'video_id': 'vIFKGFl1Cn8'},
              'score': 885.683655,
  

In [ ]:
def captions_to_embeddings(videos_captions):
    api_key = 'SrLBFEol9tRK7n7LY9FRRcHuN9QT5MLIzLmBSziT'
    # Create and retrieve a Cohere API key from os.cohere.ai
    co = cohere.Client(api_key)
    all_embeds = []
    for captions in videos_captions :
        embeds = []
        texts = []
        video_embeddings = []
        caps = captions['video_captions']
        for i in range(len(caps)):
            texts.append(caps[i]['text'])
            dic['start'] = caps[i]['start']
            dic['video_id'] = caps[i]['video_id']
            embeds.append(dic)
            if((i+1)%16 == 0):
                video_embeddings + = co.embed(texts,truncate='RIGHT'model="small").embeddings
                text = []
        if(len(texts) > 0):
            video_embeddings + = co.embed(texts,truncate='RIGHT'model="small").embeddings
            
        for i in range(len(embeds)):
            embeds[i]['embedding'] =  video_embeddings[i]
        all_embeds.append(embeds)    